In [4]:
# testing reaction_recommender class
import collections
import itertools
import libsbml
import networkx as nx
import numpy as np
import os
import pandas as pd
import pickle
import random
import re
import requests

import matplotlib.pyplot as plt
%matplotlib inline  

script_path = "/Users/woosubs/Desktop/AutomateAnnotation/reaction_recommender/reaction_recommender"
os.sys.path.append(script_path)
# import get_annotation as ga
import annotation_container as ac
# import rhea_reaction_from_chebi_species as cherhea

BASE_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/'
BIOMD_12 = 'BIOMD0000000012.xml'
DATA_DIR = os.path.join(BASE_DIR, "DATA")
RHEA_DIR = os.path.join(DATA_DIR, "rhea")
# BioModel files
BIOMODEL_DIR = os.path.join(DATA_DIR, "biomodels/curated_biomodels_31mar2021")
biomd_files = [f for f in os.listdir(BIOMODEL_DIR) if f[-4:] == ".xml"]
print(len(biomd_files))
biomd_fpaths = [os.path.join(BIOMODEL_DIR, one_file) for one_file in biomd_files]

# BiGG files
BIGG_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/DATA/bigg'
ecoli_fpath = os.path.join(BIGG_DIR, "e_coli_core.xml")
bigg_files = [f for f in os.listdir(BIGG_DIR) if f[-4:]=='.xml']
bigg_fpaths = [os.path.join(BIGG_DIR, f) for f in bigg_files]
print(len(bigg_files))
one_fpath = bigg_fpaths[0]

1000
108


In [6]:
# Nov 14, 2021: using a shortened version, meaning H and D are removed
with open(os.path.join(DATA_DIR, 'all_shortened_chebi_to_formula_19nov2021.pkl'), 'rb') as f:
  ref_shortened_chebi_to_formula = pickle.load(f)
# below is full version
with open(os.path.join(DATA_DIR, 'all_possible_chebi_to_formula_12nov2021.pkl'), 'rb') as f:
  ref_full_chebi_to_formula = pickle.load(f)
# map secondary id to primary id
with open(os.path.join(DATA_DIR, 'chebi_second2prime_8nov2021.pickle'), 'rb') as f:
  ref_second2prime_dict = pickle.load(f)
# RHEA All Ids to Bi Id (KEGG Mappable)
with open(os.path.join(RHEA_DIR, 'rhea_all2bi.pkl'), 'rb') as f:
  ref_rhea2bi = pickle.load(f)
with open(os.path.join(RHEA_DIR, 'rhea_all2master.pkl'), 'rb') as f:
  ref_rhea2master = pickle.load(f)
with open(os.path.join(RHEA_DIR, 'rhea2chebi_reference.pkl'), 'rb') as f:
  ref_rhea2chebi = pickle.load(f)

In [12]:
one_container = ac.AnnotationContainer(model_file=os.path.join(BIGG_DIR, ecoli_fpath))

In [7]:
reader = libsbml.SBMLReader()
document = reader.readSBML(ecoli_fpath)
model = document.getModel()

In [13]:
one_spec_anot = model.getSpecies('M_glc__D_e').getAnnotationString()
one_spec_anot

'<sbml:annotation xmlns:sbml="http://www.sbml.org/sbml/level3/version1/core">\n  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">\n    <rdf:Description rdf:about="#M_glc__D_e">\n      <bqbiol:is xmlns:bqbiol="http://biomodels.net/biology-qualifiers/">\n        <rdf:Bag>\n          <rdf:li rdf:resource="http://identifiers.org/bigg.metabolite/glc__D"/>\n          <rdf:li rdf:resource="http://identifiers.org/biocyc/META:Glucopyranose"/>\n          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:12965"/>\n          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:20999"/>\n          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:4167"/>\n          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:17634"/>\n          <rdf:li rdf:resource="http://identifiers.org/hmdb/HMDB00122"/>\n          <rdf:li rdf:resource="http://identifiers.org/hmdb/HMDB06564"/>\n          <rdf:li rdf:resource="http://identifiers.org/inchi_key/WQZGKKKJIJFFOK-GASJEMHNS

In [16]:
one_container.getOntologyFromString(one_spec_anot)

[('bigg.metabolite', 'glc__D'),
 ('biocyc', 'META:Glucopyranose'),
 ('chebi', 'CHEBI:12965'),
 ('chebi', 'CHEBI:20999'),
 ('chebi', 'CHEBI:4167'),
 ('chebi', 'CHEBI:17634'),
 ('hmdb', 'HMDB00122'),
 ('hmdb', 'HMDB06564'),
 ('inchi_key', 'WQZGKKKJIJFFOK-GASJEMHNSA-N'),
 ('kegg.compound', 'C00031'),
 ('kegg.drug', 'D00009'),
 ('metanetx.chemical', 'MNXM41'),
 ('sabiork', '1406'),
 ('sabiork', '1407'),
 ('seed.compound', 'cpd26821'),
 ('seed.compound', 'cpd00027')]